Objective:
- how do the treecode scale with number of particles
- how do the treecode scale with increasing number of dimensions

In [3]:
import cProfile
from treecode_helper import *

In [4]:
fid = 'cube10000'

In [5]:
particles = read_particle(fid)

In [6]:
n_crit = 10
theta = 0.2

In [7]:
def treecode(particles, n_crit, theta):
    
    # build the tree
    root = Cell(n_crit)
    root.x, root.y, root.z = 0.5, 0.5, 0.5
    root.r = 0.5
    cells = build_tree(particles, root, n_crit)
    
    # P2M
    leaves = []
    get_multipole(particles, 0, cells, leaves, n_crit)
    
    # M2M
    upward_sweep(cells)
    
    # evaluate the potentials
    eval_potential_nv(particles, cells, n_crit, theta)
    
    return particles

In [8]:
particles = treecode(particles, n_crit, theta)

#### check the result

In [9]:
phi_direct = numpy.loadtxt('test/'+fid+'_result')
phi_tree = numpy.asarray([particle.phi for particle in particles])

In [10]:
numpy.allclose(phi_tree, phi_direct, rtol=1e-03, atol=0.)

True

In [11]:
l2_err(phi_direct, phi_tree)

L2 Norm error: 6.72896532073e-05


#### run the cProfile

In [12]:
cProfile.run("treecode(particles, n_crit, theta)")

         15566550 function calls (14230792 primitive calls) in 80.318 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   80.313   80.313 <ipython-input-7-519ca4b6b8dc>:1(treecode)
        1    0.005    0.005   80.318   80.318 <string>:1(<module>)
      584    0.018    0.000    0.062    0.000 treecode_helper.py:104(split_cell)
        1    0.079    0.079    0.153    0.153 treecode_helper.py:130(build_tree)
   4344/1    0.079    0.000    0.098    0.098 treecode_helper.py:167(get_multipole)
     4343    0.064    0.000    0.067    0.000 treecode_helper.py:196(M2M)
        1    0.002    0.002    0.070    0.070 treecode_helper.py:227(upward_sweep)
1341415/10000   42.870    0.000   79.984    0.008 treecode_helper.py:270(eval_helper_nv)
  7753408   17.586    0.000   17.586    0.000 treecode_helper.py:30(distance)
        1    0.008    0.008   79.992   79.992 treecode_helper.py:314(eval_potential_nv)
     

In [13]:
n = len(particles)

In [21]:
print("direct summation operation percentage " + str(float(7753408)/n**2*100) + "%")

direct summation operation percentage 7.753408%
